# 🔬 Ensemble Evaluation - Late Fusion WavLM + WAVLM_LARGE

Questo notebook esegue la Late Fusion dei modelli WavLM Weighted e WAVLM_LARGE per valutazione su SpeechOcean762.

## Prerequisiti
- WavLM Weighted trainato
- WAVLM_LARGE trainato
- Modelli nella cartella `outputs/backup/`

## 1. Setup Ambiente

In [ ]:
# 1.1 Monta Google Drive e verifica GPU
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi --query-gpu=name,memory.total --format=csv

In [ ]:
# 1.2 Estrai progetto
import zipfile
import os

ZIP_PATH = "/content/drive/MyDrive/phonemeRef.zip"
EXTRACT_PATH = "/content"
PROJECT_DIR = "/content/DeepLearning-Phoneme"

if os.path.exists(ZIP_PATH):
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)
    # Trova cartella estratta
    extracted = [f for f in os.listdir('/content/') if os.path.isdir(f'/content/{f}') and 'Phoneme' in f]
    if extracted:
        PROJECT_DIR = f'/content/{extracted[0]}'
    print(f"✓ Estratto in {PROJECT_DIR}")
else:
    print(f"❌ File non trovato: {ZIP_PATH}")

# Cambia directory al progetto
os.chdir(PROJECT_DIR)
print(f"✓ Working directory: {os.getcwd()}")

In [ ]:
# 1.3 Installa dipendenze
!pip install -q transformers datasets evaluate jiwer soundfile librosa
!pip install -q scipy scikit-learn torchcodec
print("✓ Dipendenze installate")

## 2. Configurazione Modelli

In [ ]:
# 2.1 Path ai modelli trainati (dentro phonemeRef.zip)
import os

# ============================================================
# PATH AI MODELLI (dentro outputs/backup/)
# ============================================================
WAVLM_WEIGHT_MODEL_PATH = f"{PROJECT_DIR}/outputs/backup/wavLM_large/final_model_aug_comb_weighted"
WAVLM_LARGE_MODEL_PATH = f"{PROJECT_DIR}/outputs/backup/wavLM_large/final_model_aug_comb"

# Peso per la fusion (0.5 = media, >0.5 = più peso a WavLM)
FUSION_WEIGHT = 0.5
# ============================================================

print("📁 Configurazione Ensemble:")
print(f"   WavLM Weighted: {WAVLM_WEIGHT_MODEL_PATH}")
print(f"   WavLM Large:    {WAVLM_LARGE_MODEL_PATH}")
print(f"   Fusion Weight:  {FUSION_WEIGHT}")

# Verifica esistenza
for name, path in [("WavLM", WAVLM_WEIGHT_MODEL_PATH), ("WAVLM_LARGE", WAVLM_LARGE_MODEL_PATH)]:
    if os.path.exists(path):
        print(f"   ✓ {name}: trovato")
    else:
        print(f"   ❌ {name}: NON TROVATO!")

## 3. Esegui Late Fusion Benchmark

In [ ]:
# 3.1 Esegui Late Fusion Evaluation
import os
os.chdir(PROJECT_DIR)

print("="*60)
print("🔬 LATE FUSION BENCHMARK")
print("="*60)
print(f"WavLM: {WAVLM_WEIGHT_MODEL_PATH}")
print(f"WAVLM_LARGE: {WAVLM_LARGE_MODEL_PATH}")
print(f"Weight: {FUSION_WEIGHT}")
print("="*60)

!python scripts/evaluation/evaluate_fusion.py \
    --model-a "{WAVLM_WEIGHT_MODEL_PATH}" \
    --model-b "{WAVLM_LARGE_MODEL_PATH}" \
    --weight {FUSION_WEIGHT}

## 4. Confronto Modelli Singoli vs Ensemble

In [ ]:
# 4.1 Valutazione WavLM Weighted singolo
import os
os.chdir(PROJECT_DIR)

print("="*60)
print("🔬 VALUTAZIONE WAVLM WEIGHTED (singolo)")
print("="*60)

!python scripts/evaluation/evaluate_speechocean.py --model-path "{WAVLM_WEIGHT_MODEL_PATH}"

In [ ]:
# 4.2 Valutazione WAVLM_LARGE singolo
import os
os.chdir(PROJECT_DIR)

print("="*60)
print("🔬 VALUTAZIONE WAVLM_LARGE (singolo)")
print("="*60)

!python scripts/evaluation/evaluate_speechocean.py --model-path "{WAVLM_LARGE_MODEL_PATH}"

## 5. Sperimenta con diversi Weight

In [ ]:
# 5.1 Test diversi valori di weight
# weight = peso per modello A/WavLM (1-weight = peso per WAVLM_LARGE)
import os
os.chdir(PROJECT_DIR)

for weight in [0.3, 0.5, 0.7]:
    print("\n" + "="*60)
    print(f"🔬 FUSION con Weight = {weight}")
    print("="*60)
    
    !python scripts/evaluation/evaluate_fusion.py \
        --model-a "{WAVLM_WEIGHT_MODEL_PATH}" \
        --model-b "{WAVLM_LARGE_MODEL_PATH}" \
        --weight {weight} \
        --quiet

## 6. Salva Risultati

In [ ]:
# 6.1 Registra risultati nel benchmark tracker
# NOTA: Modifica i valori con i risultati ottenuti dal tuo run
import os
os.chdir(PROJECT_DIR)

# Esempio di comando (decommentare e modificare):
# !python scripts/evaluation/track_benchmark.py \
#     --model_name "Ensemble WavLM+WAVLM_LARGE" \
#     --architecture Ensemble \
#     --training_data "Aug_Comb" \
#     --per XX.XX \
#     --pearson 0.XXXX \
#     --auc 0.XXXX \
#     --recall 0.XXXX \
#     --notes "Late Fusion weight=0.5"

print("📊 Modifica i valori sopra con i tuoi risultati e riesegui la cella")